# In this notebook we will construct a basic interactive map using the Folium library.
#  This map will visualize the fire incidents, provided by EFFIS, that have occurred in Greece throughout the ongoing year of 2023.

## By Ilektra Karasante, 31.08.23

## Insert libraries 

In [1]:
#general
from pathlib import Path


#data
import geopandas as gpd
import pandas as pd
#display all columns
pd.set_option('display.max_columns', None)

## Folium 
import folium
from folium import plugins
import base64
from folium import IFrame
from folium.features import GeoJsonTooltip

/home/jupyter-electra/.virtualenvs/myenv/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


## Download the data from EFFIS repository: https://effis.jrc.ec.europa.eu/apps/data.request.form/ and save it locally

In [2]:
## Load the shapefile 
effis_path='insert_your_path_for_the_shapefile/Greece_fires_2023.shp'
fires = gpd.read_file(Path.home() / effis_path)
#set the crs to a geographic one 
fires=fires.to_crs('epsg:4326')

In [3]:
## View the columns available and deside which ones you want to use
fires.columns

Index(['id', 'initialdat', 'finaldate', 'area_ha', 'iso2', 'iso3', 'country',
       'admlvl1', 'admlvl2', 'admlvl3', 'admlvl5', 'map_source', 'broadleave',
       'coniferous', 'mixed_fore', 'sclerophil', 'transition', 'other_natu',
       'agricultur', 'artificial', 'other_perc', 'natura2k_p', 'noneu',
       'updated', 'area_code', 'eu_area', 'geometry'],
      dtype='object')

In [4]:
## Create columns of latitude and longitude to display the fire markers 
## Many warnings will appear because of geographic crs but that's not a issue 
fires['Latitude']=0
fires['Longitude']=0

for l, r in fires.iterrows():
    fires['Latitude'].iloc[l]= fires.geometry.centroid.y.iloc[l]
    fires['Longitude'].iloc[l]= fires.geometry.centroid.x.iloc[l]
    


/tmp/ipykernel_2308705/3627148569.py:7: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  fires['Latitude'].iloc[l]= fires.geometry.centroid.y.iloc[l]
/tmp/ipykernel_2308705/3627148569.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fires['Latitude'].iloc[l]= fires.geometry.centroid.y.iloc[l]
/tmp/ipykernel_2308705/3627148569.py:8: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  fires['Longitude'].iloc[l]= fires.geometry.centroid.x.iloc[l]
/tmp/ipykernel_2308705/3627148569.py:8: SettingWithCopyWarning: 
A value is trying to be 

In [5]:
## For the popup information I choose the variables of ignition date, final date,percentage of Natura Area that got burned,
## the local area that the fire occured and the burned areas measured in hectares 
def popup_html(row):
    i = row
    Ignition_date=fires['initialdat'].iloc[i] 
    Final_date=fires['finaldate'].iloc[i]
    Natura = fires['natura2k_p'].iloc[i].round(decimals=1) 
    Community=fires['admlvl5'].iloc[i] 
    burned_hectares = fires['area_ha'].iloc[i]

    
    left_col_color = "#19a7bd"
    right_col_color = "#f2f0d3"
    
    html = """<!DOCTYPE html>
    
<html>
<head>
</head>
    <table style="height: 126px; width: 270px;">
<tbody>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Ignition date</span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(Ignition_date) + """
</tr>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Final date</span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(Final_date) + """
</tr>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Natura Percentage Burned</span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(Natura) + """
</tr>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Community</span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(Community) + """
</tr>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">burned_hectares</span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(burned_hectares) + """
</tr>
</tbody>
</table>
</html>
"""
    return html


In [6]:
# Set location to cover Greece and the zoom you want 
location = 39, 24
map = folium.Map(location=location,zoom_start=6.3)

################################### Display the shapefiles #######################

for l, r in fires.iterrows():
    sim_geo = gpd.GeoSeries(r["geometry"])
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,name ='EFFIS Burned Area layer', style_function=lambda x: {"fillColor": "FFFFFF","color":"black"})
    geo_j.add_to(map)
    
################################### Display markers ##################################

# for i in range(0,len(fires)):
for l, r in fires.iterrows():
    html = popup_html(l)
    # iframe = branca.element.IFrame(html=html,width=510,height=280)
    popup = folium.Popup(folium.Html(html, script=True), max_width=500)
    folium.Marker(location=
        [fires.Latitude[l], fires.Longitude[l]],
        icon=folium.Icon(color='red',icon_color='yellow',icon = "fire") ,
           popup=popup
        ).add_to(map)  

############################# Pluggings ########################################

#Mini Map
mini_map = folium.plugins.MiniMap(toggle_display=True)
map.add_child(mini_map)
folium.LayerControl().add_to(map)

## Full Screen 

plugins.Fullscreen(
    position='topright',
    title='Expand me',
    title_cancel='Exit me',
    force_separate_button=True
).add_to(map)

############################## Display Map ###########################
map



In [7]:
## Save to an html file
map.save("Fires_of_Greece_2023.html")

# To Host websites (your html file) on Google Drive follow the steps below:

## 1. Upload the file to your google drive.
## 2. Visit https://www.drv.tw/ and follow the instructions 
## 3. Copy the link of your html

## You have now an online dashboard, accesible by anyone with the link
